In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai




In [ ]:
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
openai = OpenAI()


In [ ]:
system_message = "You are a helpful assistant"

In [ ]:
def message_gpt(prompt):
    messages= [{
        "role":"system","content" : system_message
    },{
        "role":"user","content": prompt
    }]

    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message.content



In [ ]:
message_gpt("Why are the cat raise call tortie")

In [ ]:
def upperCase(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [ ]:
upperCase("tyson")

In [ ]:
gr.Interface(fn=upperCase, inputs="textbox", outputs="textbox").launch()

In [ ]:
gr.Interface(fn=upperCase, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
system_message = "You are a helpful assistant that responds in markdown"

gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
).launch()

In [ ]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
system_message = "You are a helpful assistant that responds in markdown"
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [19]:

import google.generativeai as genai


# Konfigurera API-nyckeln.
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")

system_message = "You are a helpful assistant that responds in markdown"

def stream_gemini(prompt):
    # Systemmeddelandet kombineras nu direkt med användarens fråga.
    full_prompt = f"{system_message}\n\n{prompt}"
    
    # Vi skickar nu bara ett enda meddelande med rollen "user".
    messages = [
        {"role": "user", "parts": [full_prompt]}
    ]

    response = model.generate_content(
        contents=messages,
        stream=True
    )

    for chunk in response: 
        yield chunk.text



In [20]:
view = gr.Interface(
    fn=stream_gemini,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)

view.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.
